In [275]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
comments=pd.read_csv(r"../input/youtube-comments/UScomments.csv",error_bad_lines= False)

In [10]:
comments.head(11)

**SENTIMENT ANALYSIS**

In [11]:
comments.isnull().sum()

In [12]:
comments.dropna(inplace= True)

In [13]:
comments.head(40)

****Sentiment Analysis

In [ ]:
!pip install textblob 

In [ ]:
from textblob import TextBlob as tb

In [ ]:
tb("I love Logan and Jake so much and thay are so ...	").sentiment.polarity

In [ ]:
dataframe= comments[0:1000]

In [22]:
polarity=[]
for comment in comments["comment_text"]:
    try:
        polarity.append(tb(comment).sentiment.polarity)
    except:
        polarity.append(0)

In [23]:
polarity[0:11]

In [25]:
comments["polarity"]= polarity

In [26]:
comments.head(16)

**WordCloud Representation**

In [27]:
positive_comments= comments[comments["polarity"]==1]

In [32]:
positive_comments.head(11)

In [29]:
negative_comments= comments[comments["polarity"]==-1]

In [33]:
negative_comments.head(11)

In [34]:
!pip install wordcloud 

In [82]:
from wordcloud import WordCloud as wc, STOPWORDS 

In [83]:
positive_comments["comment_text"]

In [84]:
total_positive_comments= "  ".join(positive_comments["comment_text"])

In [85]:
total_positive_comments[0:100]

In [98]:
wordcloud=wc(stopwords= set(STOPWORDS), collocations = False, background_color = 'black').generate(total_positive_comments)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()


In [99]:
total_negative_comments=" ".join(negative_comments["comment_text"])

In [101]:
total_negative_comments[1:100]

In [106]:
wordcloud= wc(stopwords=set(STOPWORDS),collocations = False, background_color="black").generate(total_negative_comments)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

**EMOJI ANALYSIS**

In [107]:
!pip install emoji

In [123]:
import emoji

In [124]:
comments.head(14)

In [125]:
comment="The end though 😭👍🏻❤️"

In [126]:
comment

In [130]:
[c for c in comment if c in emoji.distinct_emoji_list(c)]

In [137]:
emoji_list=[]
for comment in comments["comment_text"]:
    for char in comment:
        if char in emoji.distinct_emoji_list(char):
            emoji_list.append(char)
        
        
           

In [138]:
len(emoji_list)

In [139]:
emoji_list[0:21]

In [141]:
from collections import Counter

In [146]:
Counter(emoji_list[0:21])

In [152]:
Counter(emoji_list).most_common(10)

In [154]:
emojis= [Counter(emoji_list).most_common(10)[i][0] for i in range(10)]

In [155]:
emojis

In [156]:
freq= [Counter(emoji_list).most_common(10)[i][1] for i in range(10)]

In [157]:
freq

In [158]:
!pip install plotly

In [160]:
import plotly.graph_objs as go
from plotly.offline import iplot

In [163]:
graph = go.Bar(x=emojis, y=freq)

In [164]:
iplot([graph])

**COLLECTING ENTIRE YOUTUBE DATA**

In [165]:
import os

In [172]:
path= r"../input/youtube-new"

In [202]:
files = os.listdir(path)

In [203]:
files

In [250]:
file1= files[0:3]

In [251]:
file1

In [252]:
file2= files[7:8]
    
    
    

In [253]:
file2


In [254]:
file3= files[10:14]

In [255]:
file3

In [256]:
file4= files[17:18]

In [257]:
file4

In [258]:
file5=files[19:20]

In [259]:
file5

In [260]:
file_csv=[*file1,*file2,*file3,*file4,*file5]


In [264]:
csv=(file_csv)
print(csv)

In [268]:
csv[0].split(".")[0][0:2]

In [278]:
full_df=pd.DataFrame()
for file in csv:
    current_df=pd.read_csv(path +"/"+ file, encoding="iso-8859-1",error_bad_lines= False)
    current_df["country"]=file.split(".")[0][0:2]
    full_df= pd.concat([full_df, current_df])

In [316]:
full_df.head(5)

In [317]:
full_df.head(5)

In [318]:
full_df["category_id"].unique()

In [319]:
cat=pd.read_csv(r"../input/category-file/category_file.txt")

In [320]:
cat

In [293]:
cat.reset_index(inplace= True)

In [294]:
cat

In [296]:
cat.columns=("Category_id","Category_name")

In [299]:
cat.set_index("Category_id", inplace= True)

In [300]:
cat

In [302]:
dct= cat.to_dict()

In [303]:
dct["Category_name"]

In [326]:
full_df["Category_name"]=full_df["category_id"].map(dct["Category_name"])

In [327]:
full_df.columns

In [328]:
full_df.head(4)

In [330]:
plt.figure(figsize=(15,10))
sns.boxplot(x="likes", y="Category_name", data= full_df)
plt.xticks(rotation="vertical")


**AUDIANCE ENGAGEMENT ANALYSIS**

In [331]:
full_df.columns

In [333]:
full_df["like_rate"]=(full_df["likes"]/full_df["views"])*100
full_df["dislike_rate"]=(full_df["dislikes"]/full_df["views"])*100
full_df["comment_count_rate"]=(full_df["comment_count"]/full_df["views"])*100




In [334]:
full_df.head(4)

In [337]:
plt.figure(figsize=(15,10))
sns.boxplot(x="Category_name",y="like_rate", data=full_df)
plt.xticks(rotation="vertical")


In [339]:
sns.regplot(data= full_df, x="views", y="likes")

In [343]:
full_df[["likes","dislikes","views"]].corr()

In [345]:
sns.heatmap(full_df[["likes","dislikes","views"]].corr(), annot= True)

**ANALYZING TRENDING VIDEOS**

In [359]:
ppp=full_df.groupby("channel_title")["video_id"].count().sort_values(ascending= False).to_frame().reset_index().rename(columns={"video_id":"total_videos"})

In [360]:
ppp

In [367]:
import plotly.express as px


In [369]:
px.bar(data_frame=ppp[0:10],x="channel_title", y="total_videos")

**Punctuations impact on views, likes, dislikes**

In [370]:
import string

In [376]:
def punc_count(x):
      return len([c for c in x if c in string.punctuation])

In [377]:
full_df["title"]

In [378]:
text=" Taylor Swift: â¦Ready for It? (Live) - SNL"

In [379]:
punc_count(text)

In [380]:
sample_data= full_df[0:100]

In [384]:
sample_data["count_punc"]=sample_data["title"].apply(punc_count)

In [385]:
sample_data.head()

In [397]:
import plotly.express as px
px.bar(data_frame= sample_data, x="count_punc", y="views")
#plt.figure(figsize=(12,10))
#sns.boxplot(x="count_punc", y="views", data= sample_data)

In [398]:
sample_data["count_punc"].corr(sample_data["views"])

In [300]:
cat